In [3]:
import numpy as np
import pyniggli
from opf_python.lat_type import lat_type
from opf_python.niggli_lat_id import niggli_id
from opf_python.test_basis import test_cell

In [11]:
def niggli_setting(A):
    """Finds the corrdinates in G6 for the lattice A. Where G6 is
    [A[:,0].A[:,0],A[:,1].A[:,1],A[:,2].A[:,2],
    2*A[:,1].A[:,2],2*A[:,2].A[:,0],2*A[:,0].A[:,1]]
    
    Args:
        A (numpy array): The lattice vectors as columns of a matrix.
        
    Returns:
        G (list): The coordinates on G6 space.
    """
    
    a = A[:,0]
    b = A[:,1]
    c = A[:,2]
    A = np.dot(a,a)
    B = np.dot(b,b)
    C = np.dot(c,c)
    xi = np.dot(b,c)
    eta = np.dot(a,c)
    zeta = np.dot(a,b)
    
    G = [A,B,C,xi,eta,zeta]
    return G

# The Idea

The idea here is to try and see if we can figure out a method to transform the symmetry preserving supercell from our basis to the users basis by using the niggli reduced cell. The workflow would be as follows:

1) identify which of the 44 niggli cells is in use (there are two triclinic cells included in those 44 which we'll disregard)

2) Find the Supercell in the niggli basis

3) Transform back to the users basis.

UC = N

S = NHCinv

# Face centered cubic

## First basis

Face centered cubic makes a good test case for the cubic cells, all cubic cells have a single unique niggli cell.

In [4]:
U = np.transpose([[0.5,0,0.5],[0.5,0.5,0],[0,0,1]])
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

Using the brute force method I've now confirmed that this niggli cell has the same symmetry preserving HNFs as our original basis choice.

In [5]:
test_cell(U)

('Users niggli number: ', 1)


True

## Second FCC basis

Let's do a second basis just to be sure.

In [6]:
U = np.transpose([[1,1,2],[1,0,1],[2,1,1]])

In [7]:
test_cell(U)

('Users niggli number: ', 1)


True

## FCC rotated basis (10 degrees about the z-axis)

In [8]:
R = [[np.cos(10.*np.pi/180.),-np.sin(10.*np.pi/180.),0],
     [np.sin(10.*np.pi/180.),np.cos(10.*np.pi/180.),0],
    [0,0,1]]
U = np.dot(R,U)

In [9]:
test_cell(U)

('Users niggli number: ', 1)


True

# Simple Cubic

In [10]:
As = [[[1,0,0],[0,1,0],[0,0,1]], [[1,0,0],[1,1,0],[0,0,1]],
        [[1,0,1],[1,1,0],[0,0,1]],[[1,1,1],[0,1,1],[1,0,1]],[[1,0,2],[0,1,0],[0,1,1]]]
As = [np.transpose(i) for i in As]

In [11]:
for U in As:
    print(U,test_cell(U))

('Users niggli number: ', 3)
(array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 1, 0],
       [0, 1, 0],
       [1, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 0, 0],
       [0, 1, 1],
       [2, 0, 1]]), True)


# Body Centered Cubic

In [12]:
As = [[[-1,1,1],[1,-1,1],[1,1,-1]],[[1,0,0],[0,1,0],[0.5,0.5,0.5]],
        [[1,0,0],[0.5,0.5,0.5],[0,0,1]],[[0.5,-0.5,0.5],[0,1,0],[0,0,1]],
        [[0,0,2],[1,-1,1],[1,1,-1]],[[-1,1,1],[2,0,0],[0,2,0]],
        [[1,-1,3],[1,-1,1],[1,1,-1]]]
As = [np.transpose(i) for i in As]

In [14]:
for U in As:
    print(test_cell(U))

('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True


# Simple Tetragonal

Now we should check a second crystal choice. Simple tetragonal cell's are another where we already have the symmetry preserving HNFs for one of the two niggli cells. We'll have to restrict ourselves to the niggli setting with lattice norms of [A,A,C] where C > A for these tests. The following 4 lattices satisfy that condition.

In [16]:
a1 = np.array([1,0,0])
a2 = np.array([0,1,0])
a3 = np.array([0,0,2])

As = [[a1+a2,a2,a3+a1],[a1+a2+a3,a2+a3,a1+a2],[a1,a2+a1,a3+a3],[-a1,-a2+a3,a3]]
As = [np.transpose(i) for i in As]

In [17]:
for U in As:
    print(test_cell(U))

('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True


## Third Basis Choice (rotated by 15 degrees)

In [18]:
R = [[np.cos(15.*np.pi/180.),-np.sin(15.*np.pi/180.),0],
     [np.sin(15.*np.pi/180.),np.cos(15.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[2])
U = np.dot(R,U)

In [19]:
test_cell(U)

('Users niggli number: ', 11)


True

## Fourth Basis Choice (rotated by 27 degrees)

In [23]:
R = [[np.cos(27.*np.pi/180.),-np.sin(27.*np.pi/180.),0],
     [np.sin(27.*np.pi/180.),np.cos(27.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[3])
U = np.dot(R,U)

In [24]:
test_cell(U)

('Users niggli number: ', 11)


True

# Simple Orthorhombic

In [3]:
As = [[[1.00000000, 0.00000000, 0.00000000],
       [0.00000000, 1.73205080, 0.00000000],
       [0.00000000, 0.00000000, 3.26598640]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [1.50000000, -1.50000000, 0.00000000]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [2.00000000, -2.00000000, 0.00000000]]]
As = [np.transpose(i) for i in As]

In [4]:
for U in As:
    print(test_cell(U,eps=1E-8))

('Users niggli number: ', 32)
True
('Users niggli number: ', 32)
True
('Users niggli number: ', 32)
True


# Conclusion

The tests so far indicate that the transformation to the Niggli cell will work. However there are still 39 cases to check and verify in addition to a more detailed study even of these systems.

What I need to do is find a way to quickly identify the lattice type, this can be accomplished via niggli faster than the current method, then verify that the supercell's retain symmetry for a larger test case.

In other words I need a far more extensive testing suite that gets wrapped into a subroutine so that it will automatically generate supercells for multiple densities for a given cell and for each supercell verify that the cell has equal or greater symmetry than the parent. If ever this isn't the case then we'll need to have it stop and report the error, i.e., let us know that the method doesn't actually work.

# Finding the rhombohedral basis

Now we need to find the 4 different rhombohedral basis cases and code up their HNFs. We should also check to see how these basis compare to our original choices when it comes to efficiency.

In [1]:
from opf_python.trig import rhom_2_4
count = 0
for i in range(501):
    count += len(rhom_2_4(i))
count

867

In [2]:
import time
start = time.time()
rhom_2_4(100000)
end = time.time()
print(end-start)

0.56950712204


In [1]:
from opf_python.trig import rhom_9
count = 0
for i in range(501):
    count += len(rhom_9(i))
count

867

In [2]:
import time
start = time.time()
rhom_9(100000)
end = time.time()
print(end-start)

0.0615818500519


In [1]:
from opf_python.trig import rhom_24
import time
start = time.time()
count = 0
for i in range(501):
    if i%100==0:
        end = time.time()
        print(i,end-start)
        start = time.time()
    count += len(rhom_24(i))
count

(0, 0.00016999244689941406)
(100, 0.005535125732421875)
(200, 0.00962209701538086)
(300, 0.01246786117553711)
(400, 0.027656078338623047)
(500, 0.02443981170654297)


867

In [2]:
import time
start = time.time()
rhom_24(100000)
end = time.time()
print(end-start)

0.151079893112


# Transformation Approach

Since solving for the HNFs in the Niggli basis is slow, 20+ times slower than in our basis choices, I'm curious if we can do something in which we map our supercells to the niggli supercell then the users supercell without having to deal with scaling or rotations. In essenc is:

S_u = N_u C_oinv H C_o C_uinv 

A symmetry preserving super cell of the users parent cell, where N_u is the niggli cell from the users basis, C_o is the transformation form our basis to the niggli cell, H is the HNF, C_u is the transformation form the users basis to the niggli cell, and S_u is the supercell.

In [6]:
from opf_python.trig import trig_spHNFs
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

In [9]:
O = np.transpose([[1,2,2],[2,1,2],[4,3,3]])
B = reduced_cell(O)
No = B.niggli
Co = B.C
#print(B.C)
#print(niggli_id(A))
#print(niggli_setting(B.niggli))
#print(np.transpose(B.niggli))

In [42]:
print(No)
Co

[[-1. -1.  1.]
 [ 1.  0.  2.]
 [ 0.  1.  2.]]


array([[ 1.,  1.,  1.],
       [-1.,  1.,  0.],
       [ 0., -1.,  0.]])

In [39]:
def find_transformations(Nu,No):
    """Finds the S and R matrices that transform Nu to No.
    
    Args:
        Nu (numpy array): The users niggli cell.
        No (numpy array): Our niggli cell.
        
    Returns:
        R (numpy array): The rotation matrix.
        S (numpy array): The scaling matrix.
        
    Raises:
        ValueError if the determinant of R is not +/-1.
    """
    
    S = np.array([[np.linalg.norm(Nu[:,0])/np.linalg.norm(No[:,0]),0,0],
                 [0,np.linalg.norm(Nu[:,1])/np.linalg.norm(No[:,1]),0],
                 [0,0,np.linalg.norm(Nu[:,2])/np.linalg.norm(No[:,2])]])
    #print("S",S)
    #print("NoS",np.transpose(np.dot(No,S)))
    R = np.dot(np.dot(No,S),np.linalg.inv(Nu),)
    #print("R",R,np.linalg.det(R))
    #print("RNu",np.dot(R,Nu))    
    
    #if not np.allclose(abs(np.linalg.det(R)),1):
    #    raise ValueError("The transformation found an "
    #                     "incorrect rotation vector.")
    return S, R

In [40]:
a1 = np.array([1,2,2])
a2 = np.array([2,1,2])
a3 = np.array([2,2,1])

As = [[a1,a2,a3],[a2+a1,a2+a3,a1+a3],[-a1,a2+a1+a1,a3-a1],[a1+a2+a3,a1+a2,a2+a3],
        [a1+a1,a2+a2,a3+a3]]
crystal_fam=3

for U in As:
    kpd = np.random.randint(10,100)
    HNFs = trig_spHNFs(kpd)
    #print(kpd,len(HNFs))
    Bu = reduced_cell(np.transpose(U))
    Cu = Bu.C
    Nu = Bu.niggli
    S, R = find_transformations(Nu,No)
    #print("Nu",np.transpose(Nu))
    #print(niggli_setting(Nu))
    for H in HNFs:
        Su = np.dot(np.dot(np.linalg.inv(R),
                           np.dot(np.dot(np.dot(O,H),Co),S)),
                    np.linalg.inv(Cu))
        #lat_name, niggli_n, lat_fam = niggli_id(Su)
        lat_fam = crystal_fam
        r = np.linalg.inv(np.transpose(U))
        g = np.linalg.inv(np.transpose(Su))
        temp = np.round(np.dot(np.linalg.inv(g),r),3)
        if lat_fam > crystal_fam or not np.allclose(temp%1,0):
            print("Failed lat_fam",lat_fam,"int?",temp%1)
            print("H",H)
            print("U",np.transpose(U))
            print("C_u",Cu)
            print("S",S)
            print("R",R)
            print("Nu",Nu)

('Failed lat_fam', 3, 'int?', array([[ 0.352,  0.352,  0.297],
       [ 0.352,  0.352,  0.297],
       [ 0.352,  0.352,  0.297]]))
('H', [[1, 0, 0], [0, 1, 0], [47, 47, 95]])
('U', array([[3, 4, 3],
       [3, 3, 4],
       [4, 3, 3]]))
('C_u', array([[ 0., -1.,  0.],
       [ 1.,  1.,  1.],
       [-1.,  0.,  0.]]))
('S', array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  1.94365063]]))
('R', array([[-0.40563494,  0.59436506,  0.59436506],
       [ 0.68873013, -0.31126987,  0.68873013],
       [ 0.68873013,  0.68873013, -0.31126987]]))
('Nu', array([[ 1.,  1.,  4.],
       [-1.,  0.,  3.],
       [ 0., -1.,  3.]]))
('Failed lat_fam', 3, 'int?', array([[ 0.8,  0.8,  0.8],
       [ 0.6,  0.6,  0.6],
       [ 0. ,  0. ,  0. ]]))
('H', [[1, 0, 0], [0, 2, 0], [19, 12, 26]])
('U', array([[-1,  4,  1],
       [-2,  5,  0],
       [-2,  6, -1]]))
('C_u', array([[ 0., -3., -1.],
       [ 0., -1.,  0.],
       [-1.,

Clearly this didn't work. Apparently, even though we can map from the niggli super cell to the users supercell this more complex evaluation fails to preserve the commensurability of the grid and the cell.

# Finding the 2 Hexagonal Niggli cells

We'll follow the same procedure as above to find and test the Hexagonal Niggli Cells.

In [7]:
A = np.transpose([[1,0,0],[-0.5,0.8660254037844386,0],[0,0,-2]])
niggli_setting(A)

[1.0, 0.99999999999999989, 4.0, 0.0, 0.0, -0.5]

In [8]:
A = np.transpose([[0,0,-0.5],[1,0,0],[-0.5,0.8660254037844386,0]])
niggli_setting(A)

[0.25, 1.0, 0.99999999999999989, -0.5, 0.0, 0.0]

In [11]:
from opf_python.hx import hex_12
count = 0
for i in range(501):
    count += len(hex_12(i))
count

1047

In [12]:
import time
start = time.time()
hex_12(100000)
end = time.time()
print(end-start)

0.381227016449


In [1]:
from opf_python.hx import hex_22
count = 0
for i in range(501):
    count += len(hex_22(i))
count

1047

In [2]:
import time
start = time.time()
hex_22(100000)
end = time.time()
print(end-start)

0.17823600769


# Finding the Second Simple Tetragonal Niggli Cell

The first niggli cell (11) for simple tetragonal was one we were already using. Now we just need the second (21).

In [4]:
A = np.transpose([[0,0,0.5],[1,0,0],[0,1,0]])
niggli_setting(A)

[0.25, 1.0, 1.0, 0.0, 0.0, 0.0]

In [7]:
from opf_python.pyniggli import reduced_cell
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 0. ,  0. ,  0.5],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ]])

In [5]:
from opf_python.stet import stet_21
count = 0
for i in range(501):
    count += len(stet_21(i))
count

1760

In [4]:
import time
start = time.time()
stet_21(100000)
end = time.time()
print(end-start)

0.0382399559021


# Finding the 4 Body Centered Tetragonal Cells

In [3]:
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

In [5]:
A = np.transpose([[-1,1,2],[1,-1,2],[1,1,-2]])
niggli_id(A)

('body centered tetragonal', 15, 4)

## Number 15

In [6]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 2.,  0.,  0.],
       [ 0.,  2.,  0.],
       [-1., -1.,  2.]])

In [1]:
from opf_python.body_tet import body_tet_15
count = 0
for i in range(501):
    count += len(body_tet_15(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_15(100000)
end = time.time()
print(end-start)

1.80545282364


## Niggli 7

In [28]:
A = np.transpose([[-1, 1, 2],[1, 1.60788, -1.55394],
                     [1.95095, -1.41625, 0.433603]])
niggli_id(A)

('body centered tetragonal', 7, 4)

In [29]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1.95095 ,  1.41625 , -0.433603],
       [ 1.      , -1.      , -2.      ],
       [ 1.95095 ,  1.19163 ,  0.879663]])

In [1]:
from opf_python.body_tet import body_tet_7
count = 0
for i in range(1,501):
    count += len(body_tet_7(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_7(100000)
end = time.time()
print(end-start)

0.460477113724


## Niggli 6

In [4]:
A = np.transpose([[-1, 1, 2],[1, 1.60788, -1.55394],
                     [1.80278, -1.47253, 0.762655]])
niggli_id(A)

('body centered tetragonal', 6, 4)

In [5]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1.      ,  1.      ,  2.      ],
       [ 1.      ,  1.60788 , -1.55394 ],
       [ 1.80278 , -1.47253 ,  0.762655]])

In [1]:
from opf_python.body_tet import body_tet_6
count = 0
for i in range(1,501):
    count += len(body_tet_6(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_6(100000)
end = time.time()
print(end-start)

1.17160606384


## Niggli 18

In [8]:
A = np.transpose([[0,0,2],[1,-2,1],
                  [-2,-1,1]])
niggli_id(A)

('body centered tetragonal', 18, 4)

In [9]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 0,  0,  2],
       [ 1, -2,  1],
       [-2, -1,  1]])

In [1]:
from opf_python.body_tet import body_tet_18
count = 0
for i in range(1,501):
    count += len(body_tet_18(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_18(100000)
end = time.time()
print(end-start)

1.24094605446


# Body Centered Ortho

## Niggli 8

In [3]:
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

## Niggli 19

In [5]:
A = np.transpose([[-0.5,1,1.5],[0.5,-1,1.5],[0.5,1,-1.5]])
niggli_id(A)

('body centered orthorhombic', 19, 5)

In [6]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1. ,  0. ,  0. ],
       [-0.5, -1. ,  1.5],
       [-0.5,  1. ,  1.5]])

In [1]:
from opf_python.body_ortho import body_ortho_19
count = 0
for i in range(1,501):
    if i%100==0:
        print(i)
    count += len(body_ortho_19(i))
count

100
200
300
400
500


14578

## Niggli 8

In [45]:
A = np.transpose([[1, 1, 2],[1.41144,0.0885622,-2],
                 [-1.99868,1.21232,-0.731822]])
niggli_id(A)

('body centered orthorhombic', 8, 5)

In [46]:
B = reduced_cell(A,eps=1E-4)
np.transpose(B.niggli)

array([[ 1.       ,  1.       ,  2.       ],
       [ 1.41144  ,  0.0885622, -2.       ],
       [-1.99868  ,  1.21232  , -0.731822 ]])

In [1]:
from opf_python.body_ortho import body_ortho_8
count = 0
for i in range(1,501):
    count += len(body_ortho_8(i))
count

15972

In [2]:
import time
start = time.time()
body_ortho_8(100000)
end = time.time()
print(end-start)

13.5245568752


# Niggli cells to debug!

In [ ]:
A = np.transpose([[1, 1, 2],[1.36603,-0.366025,-2],
                 [-1.67711,1.52701,-0.924951]])
A = np.transpose([[-1.67711 ,  1.52701 , -0.924951],
       [-0.68892 , -2.160985,  0.924951],
       [ 1.      ,  1.      ,  2.      ]])
